Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Tear it all down
Use this notebook to clean up the web service, image, model and the AKS cluster created by the tutorial.

In [ ]:
from azureml.core.compute import AksCompute
from azureml.core.image import Image
from azureml.core.model import Model
from azureml.core.webservice import AksWebservice
from azureml.core.workspace import Workspace
from dotenv import get_key, find_dotenv
from utilities import get_auth


In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

Let's get the workspace information.

In [ ]:
ws = Workspace.from_config(auth=get_auth(env_path))
print(ws.name, ws.resource_group, ws.location, sep="\n")

Let's retrieve the web service to delete.

In [ ]:
aks_service_name = get_key(env_path, 'aks_service_name')
aks_service = AksWebservice(ws, name=aks_service_name)
print(aks_service.name, aks_service.tags)

Let's retrieve the image to delete.

In [ ]:
image_name = get_key(env_path, 'image_name')
image_version = int(get_key(env_path, 'image_version'))
image = Image(ws, name=image_name, version=image_version)
print(image.name, image.version)

Let's retrieve the model to delete.

In [ ]:
model_name = 'question_match_model'
model_version = int(get_key(env_path, 'model_version'))
model = Model(ws, name=model_name, version=model_version)
print(model.name, model.version)

Let's retrieve the AKS compute to delete.

In [ ]:
aks_name = get_key(env_path, 'aks_name')
aks_target = AksCompute(ws, name=aks_name)
print(aks_target.name, aks_target.get_status())

Delete the service, image and model. 

In [ ]:
%%time
aks_service.delete()
image.delete()
model.delete()

Let's delete the AKS compute from the workspace. Since we created the cluster through AML, the corresponding cloud based objects will also be deleted. If the custer was created externally and attached to the workspace, the below would raise an exception and nothing will be changed.

In [ ]:
%%time
aks_target.delete()

If you also would like to delete the workspace and all experiments in it, you can use the following.

In [ ]:
%%time
ws.delete()

Finally, you can delete the resource group with the following.

In [ ]:
resource_group = get_key(env_path, 'resource_group')

In [ ]:
!az group delete --yes --name $resource_group